## Project

In [40]:
import pandas as pd
import numpy as np
import requests
import os
import xarray as xr
from netCDF4 import Dataset
import tempfile
from scipy.spatial.distance import cdist
from tqdm import tqdm
from math import radians, cos, sin, asin, sqrt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import ensemble
import matplotlib.pyplot as plt

### 1. Data Collection

#### 1.1 Biomass data

Spatio-temporal data for walleye pollock in the Gulf of Alaska extracted from the NOAA Fisheries website: https://apps-st.fisheries.noaa.gov/dismap/DisMAP.html#single-species-distributions

In [7]:
wp_gulf_alaska = pd.read_csv("wp_gulf_alaska.csv")
wp_gulf_alaska.head()

,HaulID,Stratum,LAT,LON,Depth,Year,wtcpue
0,021-198403-001,350,55.98767,-134.59517,346,1984,0.0000
1,021-198403-002,251,55.63200,-134.22900,225,1984,4.7482
2,021-198403-003,151,55.21450,-133.88733,117,1984,1.0413
3,021-198403-006,151,55.13300,-133.87100,141,1984,1.3450
4,021-198403-007,151,55.12550,-134.00167,170,1984,0.2030


Columns:
- HaulID / Stratum: characterizes the survey expedition and catch throw.
- LAT / LON: Latitude and Longitude of survey catch
- Depth: Sea depth measured in meters below sea level.
- Year: No other timestamp is provided. However, documentation specify that all surveys have been conducted during summer months.
- wtcpue: survey catch measured in kg per ha.


In [8]:
wp_gulf_alaska = wp_gulf_alaska.rename(columns={"HaulID": "haul_id", "Stratum": "stratum", "LAT": "lat", "LON": "lon", "Depth":'depth', "Year": "year"})
wp_gulf_alaska['year'] = wp_gulf_alaska['year'].astype('int64')
wp_gulf_alaska.head()

,haul_id,stratum,lat,lon,depth,year,wtcpue
0,021-198403-001,350,55.98767,-134.59517,346,1984,0.0000
1,021-198403-002,251,55.63200,-134.22900,225,1984,4.7482
2,021-198403-003,151,55.21450,-133.88733,117,1984,1.0413
3,021-198403-006,151,55.13300,-133.87100,141,1984,1.3450
4,021-198403-007,151,55.12550,-134.00167,170,1984,0.2030


#### 1.2 Oceanographic data

We web-scrape temperature and salinity geospatial data from HyCOM (Hybrid Coordinate Ocean Model) using their NCSS request URL. Since there is too much data, we have decided to only get a sample from the first day of each summer month (July 1st, August 1st and September 1st). Of course, including more points should improve our model. Then again, we don't have a timestamp associated with each survey, just the year.

We will extract data from this geographic region:
- North limit: 60.32
- South limit: 52.41
- West limit: -170
- East limit: -132.5

Unfortunately, we do not have data prior to 1995. 

In [15]:
# Define the list of years and months
year_list = [1996, 1999, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019]
month_list = [7, 8, 9]
this_folder = os.getcwd()
data_folder = this_folder + '/temp' # '\\temp' on a windows machine

if not os.path.exists('ocean_data.csv'):
  # create data folder if it doesn't exists 
  if not os.path.exists(data_folder):
    os.mkdir(data_folder)
  
  # Create an empty DataFrame to store the results
  df_all = pd.DataFrame()

  # Loop over each desired year and month, and add a progress bar
  for year in year_list:
      for month in tqdm(month_list, desc=f'{year}'):
          # Define the date string
          date_str = f'{year}-{month:02d}-01T09:00:00Z'
          # Define the URL with the fixed parameters
          if year <=2015:
              url = "https://ncss.hycom.org/thredds/ncss/GLBv0.08/expt_53.X/data/"+str(year)+"?var=salinity_bottom&var=water_temp_bottom&north=60.32&west=-170&east=-132.5&south=52.41&horizStride=1&vertCoord=&accept=netcdf4"
          elif year == 2017:
              url = "https://ncss.hycom.org/thredds/ncss/GLBv0.08/expt_57.7?var=salinity_bottom&var=water_temp_bottom&north=60.32&west=-170&east=-132.5&south=52.41&horizStride=1&vertCoord=&accept=netcdf4"
          else:
              url = "https://ncss.hycom.org/thredds/ncss/GLBv0.08/expt_93.0/ts3z?var=salinity_bottom&var=water_temp_bottom&north=60.32&west=-170&east=-132.5&south=52.41&horizStride=1&vertCoord=&accept=netcdf4"
          
          my_file = os.path.join(this_folder, 'temp', f'example-{date_str[:10]}.nc4')
          # Add the date parameter to the URL
          url_day = f'{url}&time={date_str}'
          # Download the data and save it to a file
          downloaded_obj = requests.get(url_day)
          with open(my_file, "wb") as file:
              file.write(downloaded_obj.content)
              del downloaded_obj
          # Open the NetCDF file and convert it to a pandas DataFrame
          ds = xr.open_dataset(my_file)
          df_temp = ds.to_dataframe().reset_index()
          # Append the DataFrame to the overall DataFrame
          df_all = pd.concat([df_all, df_temp], ignore_index=True)

  df_all.to_csv('ocean_data.csv', index=False)

2019: 100%|█████████████████████████████████████████████████████████████| 3/3 [01:56<00:00, 39.00s/it]


In [3]:
ocean_data_df = pd.read_csv('ocean_data.csv')
ocean_data_df.head()

,time,lat,lon,salinity_bottom,water_temp_bottom
0,1996-07-01 09:00:00,52.400002,-170.000000,34.223000,3.535999
1,1996-07-01 09:00:00,52.400002,-169.919983,34.287003,3.223999
2,1996-07-01 09:00:00,52.400002,-169.839966,34.330000,3.050999
3,1996-07-01 09:00:00,52.400002,-169.760010,34.323000,3.139000
4,1996-07-01 09:00:00,52.400002,-169.679993,34.324000,3.146000


In [4]:
ocean_data_df['time'] = pd.to_datetime(ocean_data_df['time'])
ocean_data_df['year'] = ocean_data_df['time'].dt.strftime('%Y').astype(int)
ocean_data_df.head()

,time,lat,lon,salinity_bottom,water_temp_bottom,year
0,1996-07-01 09:00:00,52.400002,-170.000000,34.223000,3.535999,1996
1,1996-07-01 09:00:00,52.400002,-169.919983,34.287003,3.223999,1996
2,1996-07-01 09:00:00,52.400002,-169.839966,34.330000,3.050999,1996
3,1996-07-01 09:00:00,52.400002,-169.760010,34.323000,3.139000,1996
4,1996-07-01 09:00:00,52.400002,-169.679993,34.324000,3.146000,1996


In [9]:
min_lat = wp_gulf_alaska['lat'].min()
max_lat = wp_gulf_alaska['lat'].max()
min_lon = wp_gulf_alaska['lon'].min()
max_lon = wp_gulf_alaska['lon'].max()
print(min_lat)
print(max_lat)
print(min_lon)
print(max_lon)

# min_lat = ocean_data_df['lat'].min()
# max_lat = ocean_data_df['lat'].max()
# min_lon = ocean_data_df['lon'].min()
# max_lon = ocean_data_df['lon'].max()
# print(min_lat)
# print(max_lat)
# print(min_lon)
# print(max_lon)

52.41715
60.3205
-169.99135
-132.6795


In [10]:
small_ocean_data_df = ocean_data_df[ocean_data_df['lat'] > (min_lat - 2)]
small_ocean_data_df = small_ocean_data_df[small_ocean_data_df['lat'] < (max_lat + 2)]

small_ocean_data_df = small_ocean_data_df[small_ocean_data_df['lon'] > (min_lon - 2)]
small_ocean_data_df = small_ocean_data_df[small_ocean_data_df['lon'] < (max_lon + 2)]
small_ocean_data_df = small_ocean_data_df.dropna()

In [11]:
# define the precision of the rounding
precision = 0
# round the 'lat' and 'lon' columns
small_ocean_data_df['lat'] = small_ocean_data_df['lat'].round(precision)
small_ocean_data_df['lon'] = small_ocean_data_df['lon'].round(precision)
# group the dataframe by the rounded 'lat' and 'log' columns, and calculate the mean of the other columns
grouped_ocean_data = small_ocean_data_df.groupby(['lat', 'lon', 'year'], as_index=False).mean()
grouped_ocean_data

/tmp/ipykernel_25141/1135101294.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_ocean_data = small_ocean_data_df.groupby(['lat', 'lon', 'year'], as_index=False).mean()


,lat,lon,year,salinity_bottom,water_temp_bottom
0,52.0,-170.0,1996,34.150191,3.664523
1,52.0,-170.0,1999,34.142493,3.640523
2,52.0,-170.0,2003,34.152730,3.613444
3,52.0,-170.0,2005,34.167652,3.609475
4,52.0,-170.0,2007,34.163746,3.523126
...,...,...,...,...,...
3205,60.0,-140.0,2009,31.980448,9.603528
3206,60.0,-140.0,2011,32.032048,9.629771
3207,60.0,-140.0,2013,32.180358,9.196933
3208,60.0,-140.0,2015,32.068981,10.548961


#### 1.3 Merging the two data sets

In [12]:
grouped_ocean_data['ocean_data_id'] = grouped_ocean_data.index 
grouped_ocean_data.head()

,lat,lon,year,salinity_bottom,water_temp_bottom,ocean_data_id
0,52.0,-170.0,1996,34.150191,3.664523,0
1,52.0,-170.0,1999,34.142493,3.640523,1
2,52.0,-170.0,2003,34.152730,3.613444,2
3,52.0,-170.0,2005,34.167652,3.609475,3
4,52.0,-170.0,2007,34.163746,3.523126,4


In [14]:
wp_gulf_alaska = wp_gulf_alaska[wp_gulf_alaska['year'].isin(grouped_ocean_data.year.unique())]
sub_wp_gulf_alaska = wp_gulf_alaska
sub_wp_gulf_alaska.head()

,haul_id,stratum,lat,lon,depth,year,wtcpue
3104,023-199601-003,210,52.59615,-169.4421,235,1996,9.0793
3105,023-199601-004,111,52.78445,-168.7133,109,1996,0.3296
3106,023-199601-005,10,52.85188,-168.6160,95,1996,646.7679
3107,023-199601-006,10,52.98299,-168.2722,106,1996,256.1357
3108,023-199601-007,210,52.96844,-167.5347,227,1996,3.0989


In [15]:
def dist(lat1, lon1, lat2, lon2):
    """
    https://medium.com/analytics-vidhya/finding-nearest-pair-of-latitude-and-longitude-match-using-python-ce50d62af546
    """
    # convert decimal degrees to radians 
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371 * c
    return km

def find_nearest(lat, lon, year):
  year_ocean_data_df = grouped_ocean_data[grouped_ocean_data['year'] == year]
  
  # mini filtering on lat lng
  # year_ocean_data_df = year_ocean_data_df[(abs(year_ocean_data_df["lat"] - lat) < 10) & (abs(year_ocean_data_df["lon"] - lon) < 10)]
  
  distances = year_ocean_data_df.apply(
      lambda row: dist(lat, lon, row['lat'], row['lon']), 
      axis=1)
  return year_ocean_data_df.loc[distances.idxmin(), 'ocean_data_id']

sub_wp_gulf_alaska['ocean_data_id'] = sub_wp_gulf_alaska.apply(
    lambda row: find_nearest(row['lat'], row['lon'], row['year']), 
    axis=1)

merge_df = pd.merge(sub_wp_gulf_alaska, grouped_ocean_data, on='ocean_data_id', how='left', suffixes=('_wp', '_ocean'))

merge_df.head()

,haul_id,stratum,lat_wp,lon_wp,depth,year_wp,wtcpue,ocean_data_id,lat_ocean,lon_ocean,year_ocean,salinity_bottom,water_temp_bottom
0,023-199601-003,210,52.59615,-169.4421,235,1996,9.0793,400,53.0,-169.0,1996,33.791651,3.926612
1,023-199601-004,111,52.78445,-168.7133,109,1996,0.3296,400,53.0,-169.0,1996,33.791651,3.926612
2,023-199601-005,10,52.85188,-168.6160,95,1996,646.7679,400,53.0,-169.0,1996,33.791651,3.926612
3,023-199601-006,10,52.98299,-168.2722,106,1996,256.1357,410,53.0,-168.0,1996,33.666818,3.898750
4,023-199601-007,210,52.96844,-167.5347,227,1996,3.0989,410,53.0,-168.0,1996,33.666818,3.898750


# Boosting

In [25]:
# correct X's? what is depth? which lat / lon to use? 
y = merge_df['wtcpue']
X = merge_df[['lat_wp', 'lon_wp', 'depth','salinity_bottom','water_temp_bottom']]

# split 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=1)

In [53]:
# parameters = {
#     "loss":["deviance"],
#     "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
#     "min_samples_split": np.linspace(0.1, 0.5, 12),
#     "min_samples_leaf": np.linspace(0.1, 0.5, 12),
#     "max_depth":[3,5,8],
#     "max_features":["log2","sqrt"],
#     "criterion": ["friedman_mse",  "mae"],
#     "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
#     "n_estimators": [10]
# }

params = {
    "n_estimators": [10],
    "max_depth":[3,5,8],
    "min_samples_split": [0.1, 0.5],
    "learning_rate": [0.01, 0.025],
    "loss": ["squared_error"],
}

#mod = ensemble.GradientBoostingRegressor(**params).fit(X_train, y_train)

In [ ]:
mod_1 = ensemble.GradientBoostingRegressor()
reg = GridSearchCV(mod_1, parameters)
reg.fit(X_train, y_train)

In [ ]:
df=pd.DataFrame.from_dict(reg.cv_results_)

Boosting: 
    - predict biomass on 2019 
    - using lat / long / depth / salinity / temperature 
    
Assignments:

    - Eli : FAQ / Sans
    - Tommy : aggregate data / join frames 
    - Will : model fit / predict 
    - Alonso : gathered data 
    - Sans : math